In [ ]:
import pybullet as b
import time
import pybullet_data

import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, rmp_tree_print, node_array, ProjectionNode, PositionProjection, FrameProjection
import rmpflow.rmp.rmp_leaf as leaves

# constants
DT = 0.001 #s
urdf_file = "./assets/kuka_iiwa.urdf"

# import rmp tree
root, links, robot = rmp_tree_from_urdf(urdf_file)

# attach frame projector and position attractor
proj_node = PositionProjection("proj_xp_lbr_iiwa_link_7", links["lbr_iiwa_link_7"])
target_pos = np.array([0.5,0.5,0.5])
atrc = leaves.PosAttractorUni("pos_attractor", proj_node, target_pos.transpose(), gain=1600, eta=80)

# attach frame projector and orientation projector
# proj_node = RotationProjection("proj_xq_lbr_iiwa_link_7", links["lbr_iiwa_link_7"])
# target_pos = np.array([0.5,0.5,0.5])
# atrc = leaves.GoalAttractorUni("pos_attractor", proj_node, target_pos.transpose(), gain=1600, eta=80)

# tree printing
rmp_tree_print(root)

# setup physics
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath())
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
plane_id = b.loadURDF("plane.urdf")
start_pos = [0,0,0]
start_ori = b.getQuaternionFromEuler([0,0,0])
robot_id = b.loadURDF(urdf_file,start_pos, start_ori, flags=(b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS))
n_dof = b.getNumJoints(robot_id)
joints_id = range(n_dof) 

# setup configuration
b.setJointMotorControlArray(robot_id, joints_id, b.VELOCITY_CONTROL, forces=[0] * n_dof)

# simulate
for i in range(0, 10000):
    # read state feedback
    joint_states = b.getJointStates(robot_id, joints_id)
    q = np.array([joint_states[i][0] for i in joints_id])
    dq = np.array([joint_states[i][1] for i in joints_id])
    ddq, mass_matrix, h = root.solve(q,dq)

    # compute torque and dynamics
    tau = (mass_matrix.dot(ddq) + h).flatten()

    # command system:
    for i in joints_id:
        b.setJointMotorControl2(robot_id, i, b.TORQUE_CONTROL, force=tau[i])

    # sleep
    b.stepSimulation()
    time.sleep(DT)

# disconnect
b.disconnect()

`- root
   |- proj_q_lbr_iiwa_link_1
   |  `- lbr_iiwa_link_1
   |- proj_q_lbr_iiwa_link_2
   |  `- lbr_iiwa_link_2
   |- proj_q_lbr_iiwa_link_3
   |  `- lbr_iiwa_link_3
   |- proj_q_lbr_iiwa_link_4
   |  `- lbr_iiwa_link_4
   |- proj_q_lbr_iiwa_link_5
   |  `- lbr_iiwa_link_5
   |- proj_q_lbr_iiwa_link_6
   |  `- lbr_iiwa_link_6
   `- proj_q_lbr_iiwa_link_7
      `- lbr_iiwa_link_7
         `- proj_xp_lbr_iiwa_link_7
            `- pos_attractor


### Rotation and quaternion feedback computations

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R

r_curr = R.from_quat([0, 0, np.sin(np.pi/4), np.cos(np.pi/4)]) # x, y, z, w
r_des = R.from_quat([0, np.sin(np.pi/16), 0, np.cos(np.pi/16)])*r_curr # apply additional rotation along inertial y axis

# rotation error using Rcurr cross Rdes (curr - des)
r_curr_mat = r_curr.as_matrix()
r_des_mat = r_des.as_matrix()
c1 = r_curr_mat[:,0]
c2 = r_curr_mat[:,1]
c3 = r_curr_mat[:,2]
d1 = r_des_mat[:,0]
d2 = r_des_mat[:,1]
d3 = r_des_mat[:,2]
delta_phi = -1.0 / 2.0 * (np.cross(c1,d1) + np.cross(c2,d2) + np.cross(c3,d3))
print("From Rc^T Rd, phi = {}".format(delta_phi))

# rotation error as quaternion, similar to dR = Rdes^T R
# see "10.1109/ICRA.2011.5980556" and "7-DOF Redundant Manipulator Control using Quaternion Feedbackbased on Virtual Spring-Damper Hypothesis"
r_des_curr = r_des * r_curr.inv()
q_des_curr = r_des_curr.as_quat()
delta_phi = -2 * q_des_curr[3] * q_des_curr[:-1]
print("From quaternion feedback, phi = {}".format(delta_phi))


From Rc^T Rd, phi = [-1.47308202e-17 -3.82683432e-01 -4.70600817e-17]
From quaternion feedback, phi = [ 0.         -0.38268343  0.        ]
